# From LLM to agent

In this workshop, we will incrementally build a powerful agent-based chatbot.

This notebook will guide you through the steps, but some parts are left as exercises for you to complete.

> REMEMBER: The notebook is just there to experiment; the end result needs to be present in `chat_app.py` to be able to interactively chat.


## Setting up the basic chatbot

We will begin by creating a basic chatbot using the `ChatGoogleGenerativeAI` model.

> REMEMBER: The use of the Google API is configured by setting `GOOGLE_API_KEY=...` in a `.env` file in the root of the project.
> To get an API key: https://aistudio.google.com/app/apikey

### Exercise:

- Open the file `chat_app.py`.
- Review the code and understand how the `on_message` function streams responses from the LLM.
- Run the chatbot using the `main()` function and test it with some sample inputs.
- Consider what the `AsyncLangchainCallbackHandler` does to monitor what's going on

### Questions to consider:

- How does the `astream` method work?
- What is the role of `RunnableConfig`?


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-04-17")

## From LangChain to LangGraph

To start building a proper agent, we need to migrate from LangChain to LangGraph.
This will provide us with a thread-based message history with minimal configuration (the ability to ask follow-up questions).
Ensure you have the proper dependencies (`uv add langgraph`).

Check https://langchain-ai.github.io/langgraph/agents/agents/#basic-configuration

> TIP: Add the current date into the prompt, as it will often think of the cutoff date as 'today'


In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm=llm, ...)

## Receiving responses from the agent

LangGraph provides flexibility in how you can handle responses from the LLM. You can choose between implementing an **async streaming API** for a better user experience (as if the LLM types back) or processing responses synchronously for simplicity.

### Async Streaming API

The async streaming API allows you to stream responses incrementally as they are generated. This is particularly useful for creating a more interactive and responsive chatbot. To implement this:

1. Use the `astream` method provided by LangGraph.
2. Properly `await` the necessary coroutines.

Example:

```python
async for msg, metadata in agent.astream(
    dict(messages="Some query"),
    stream_mode="messages",
    config=RunnableConfig(
        callbacks=[cl.AsyncLangchainCallbackHandler()],
        configurable=...,
    ),
):
    print(f"Msg: {msg}")
    print(f"Metadata: {metadata}")
```

### Synchronous API

If simplicity is your priority, you can process the response in one go. This approach is easier to implement but may result in a slight delay before the user sees the response.

Example:

```python
response = agent.invoke(dict(messages="Some query"))
print(response)
```


## Adding memory and checkpointing

Use `InMemorySaver` to checkpoint (remember) conversations.

https://langchain-ai.github.io/langgraph/agents/memory/#short-term-memory

It is crucial here to add a thread ID into the mix at this point, this ensure a proper chat history across user sessions

- Check https://docs.chainlit.io/integrations/langchain#with-langgraph how to do this in Chainlit


In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

## Add tools

- Start by adding a dummy tool (like `get_weather` as in the LangGraph tutorial, or `add = operator.add` to calculate addition)
- Add a web search tool; DuckDuckGo is free to use, check https://python.langchain.com/docs/integrations/tools/ddg/
- Look into more useful tools: https://langchain-ai.github.io/langgraph/agents/tools/#prebuilt-tools
- Implement your own: https://langchain-ai.github.io/langgraph/agents/tools/#define-simple-tools


In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()